In [135]:
import numpy as np
from timeit import default_timer as timer


In [3]:
a = np.random.rand(3,4)

In [5]:
a

array([[0.98939305, 0.73756218, 0.09407944, 0.19303971],
       [0.47259369, 0.95283369, 0.52368495, 0.40698243],
       [0.84150857, 0.74079227, 0.7060555 , 0.24441571]])

In [8]:
np.amax(a, axis=1).shape

(3,)

In [13]:
ind = np.argmax(a, axis=1)

In [14]:
ind

array([0, 1, 0])

In [11]:
tar = np.random.randint(0,10,(4,7))

In [12]:
tar

array([[3, 3, 0, 5, 7, 5, 6],
       [5, 6, 0, 4, 0, 9, 7],
       [8, 2, 1, 6, 7, 8, 3],
       [7, 1, 0, 8, 3, 5, 6]])

In [17]:
tar[ind]

array([[3, 3, 0, 5, 7, 5, 6],
       [5, 6, 0, 4, 0, 9, 7],
       [3, 3, 0, 5, 7, 5, 6]])

In [18]:
c = np.array([True, True, False, True])

In [24]:
lb1 = a<0.5
lb2 = a<0.2

In [22]:
a<0.5

array([[False, False,  True,  True],
       [ True, False, False,  True],
       [False, False, False,  True]])

In [23]:
a<0.2

array([[False, False,  True,  True],
       [False, False, False, False],
       [False, False, False, False]])

In [25]:
lb1 & ~lb2

array([[False, False, False, False],
       [ True, False, False,  True],
       [False, False, False,  True]])

In [37]:
def sample_w_padding(num_elements, num_needed):
    if num_elements >= num_needed:
        return np.random.choice(num_elements, size=num_needed, replace=False)
    else:
        choice = np.arange(num_elements)
        repeat = np.random.choice(num_elements, size=num_needed-num_elements, replace=True)
        return np.concatenate((choice, repeat))
    

In [34]:
def sample_bg(num_needed, bg_easy_num, bg_hard_num, bg_hard_ratio):
    bg_easy_choice = np.array([])
    bg_hard_choice = np.array([])

    if bg_easy_num==0:
        bg_hard_choice = sample_w_padding(bg_hard_num, num_needed)
    elif bg_hard_num==0:
        bg_easy_choice = sample_w_padding(bg_easy_num, num_needed)
    else:
        bg_hard_sample_num = np.floor(bg_hard_ratio * num_needed).astype(int)
        bg_hard_choice = sample_w_padding(bg_hard_num, bg_hard_sample_num)
        bg_easy_choice = sample_w_padding(bg_easy_num, num_needed-bg_hard_sample_num)

    return np.sort(bg_easy_choice), np.sort(bg_hard_choice)

In [92]:
def sample_fg(num_needed, fg_cr1_num, fg_cr2_num):
    choice = sample_w_padding(fg_cr1_num+fg_cr2_num, num_needed)
    fg_cr1_choice = choice[choice<fg_cr1_num]
    fg_cr2_choice = choice[choice>=fg_cr1_num]-fg_cr1_num
    return np.sort(fg_cr1_choice), np.sort(fg_cr2_choice)

In [38]:
sample_bg(13,5,5, 0.5)


(array([0, 0, 1, 2, 3, 3, 4]), array([0, 0, 1, 2, 3, 4]))

In [32]:
iou = np.random.rand(10)
print(iou)

[0.44685359 0.28235285 0.55486732 0.00765685 0.83655893 0.57083364
 0.72634309 0.88255379 0.2529064  0.65665343]


In [39]:
w = np.where(iou<0)[0]
print(w)

[]


In [53]:
w

array([], dtype=int64)

In [100]:
iou[w]

array([0.76530789, 0.98037902, 0.959433  , 0.63048538, 0.97678681,
       0.94282223])

In [8]:
c=np.arange(10)

In [9]:
c[1:4:2]

array([1, 3])

In [25]:
a = np.array([1,2,3,4,5])

In [23]:
def change(a):
    a=a.copy()
    a[0]=0

In [26]:
change(a)
print(a)

[1 2 3 4 5]


In [54]:
d = np.array([],dtype=int)

In [55]:
d

array([], dtype=int64)

In [56]:
def expand_bbox(box, delta):
    '''
    expand bounding box with given distance in all directions
    input (8, 3) corners coordinates
    '''

    direction = np.sign(box)

    direction[direction == 0] = 1
    
    newbox = box + direction * delta

    return newbox

def label2corners(label, expand = False, delta = 1):
    '''
    Task 1
    input
        label (N,7) 3D bounding box with (x,y,z,h,w,l,ry)
    output
        corners (N,8,3) corner coordinates in the rectified reference frame
    '''
    corners = []
    for bbox in label:
        h = bbox[3]
        w = bbox[4]
        l = bbox[5]
        x = bbox[0]
        y = bbox[1]
        z = bbox[2]
        ry = bbox[6]

        '''
              1 -------- 0
             /|         /|
            2 -------- 3 .
            | |        | |
            . 5 -------- 4
            |/         |/
            6 -------- 7
        ''' 
        corner_3d = np.array([[l/2, -h,  w/2],
                    [-l/2, -h,  w/2],
                    [-l/2, -h, -w/2],
                    [ l/2, -h, -w/2],
                    [ l/2,  0,  w/2],
                    [-l/2,  0,  w/2],
                    [-l/2,  0, -w/2],
                    [ l/2,  0, -w/2]])

        
        #corner_3d = np.array(corner_3d)

        if expand:
            corner_3d = expand_bbox(corner_3d, delta = delta)
        corner_3d = np.hstack([corner_3d, np.ones((8,1))])
        corner_3d = corner_3d.T

        cos_ry = np.cos(ry)
        sin_ry = np.sin(ry)
        T = [[ cos_ry, 0, sin_ry, x],
            [0,       1, 0,      y],
            [-sin_ry, 0, cos_ry, z],
            [0,       0, 0,      1]]
        T = np.array(T)

        corner_3d = np.dot(T, corner_3d)
        corner_3d = corner_3d.T
        corner_3d = corner_3d[:, :3]
        corners.append(corner_3d)
    corners = np.array(corners)
    assert corners.shape == (len(label), 8, 3)

    return corners

In [114]:
def enlarge_box(box, extention):
    '''
    input
        box (N,7) 3D bounding box label (x,y,z,h,w,l,ry)
        extention (float)  extend the bounding box by extension on all sides in meters
    output
        extended_box (N,7) extended bounding box label
    '''
    box[:, 3:6] += 2 * extention
    return box

def points_in_box(xyz, box):
    '''
    input
        xyz (N,3) point coordinates in rectified reference frame
        box (7,) 3d bounding box label (x,y,z,h,w,l,ry)
    output
        flag (bool) true if point is in bounding box
    '''
    h = box[3]
    w = box[4]
    l = box[5]
    x = box[0]
    y = box[1]
    z = box[2]
    ry = box[6]

    cos_ry = np.cos(ry)
    sin_ry = np.sin(ry)
    T = [[ cos_ry, 0, sin_ry, x],
        [0,       1, 0,      y],
        [-sin_ry, 0, cos_ry, z],
        [0,       0, 0,      1]]
    T = np.array(T)
    C = T[:3, :3]
    r = T[:3,  3]
    T_inv = np.zeros((4,4))
    T_inv[:3,:3] = C.T
    T_inv[:3, 3] = np.dot(-C.T, r)
    T_inv[3,  3] = 1
    print(T_inv)

    xyz = np.hstack([xyz, np.ones((len(xyz),1))]).T
    xyz_prime = np.dot(T_inv, xyz)
    xyz_prime = xyz_prime.T
    xyz_prime = xyz_prime[:, :3]

    flag = (xyz_prime[:,0] >= -l/2) & (xyz_prime[:,0] <= l/2) & \
           (xyz_prime[:,1] <= 0) & (xyz_prime[:,1] >= -h) & \
           (xyz_prime[:,2] >= -w/2) & (xyz_prime[:,2] <= w/2)
    return np.where(flag)[0]

In [70]:
label = np.array([[0,0,0,np.sqrt(2),2,np.sqrt(2),np.pi/4]])
label = enlarge_box(label, 1)
cns = label2corners(label)
print(cns)

[[[ 2.41421356e+00 -4.00000000e+00  1.00875766e-16]
  [-8.86511593e-17 -4.00000000e+00  2.41421356e+00]
  [-2.41421356e+00 -4.00000000e+00 -1.00875766e-16]
  [ 8.86511593e-17 -4.00000000e+00 -2.41421356e+00]
  [ 2.41421356e+00  0.00000000e+00  1.00875766e-16]
  [-8.86511593e-17  0.00000000e+00  2.41421356e+00]
  [-2.41421356e+00  0.00000000e+00 -1.00875766e-16]
  [ 8.86511593e-17  0.00000000e+00 -2.41421356e+00]]]


In [116]:
xyz = np.array([[2,1,2],
                [2,-3,2],
                [2.13,-1,2.13]
])
label = np.array([2,-1,2,2,np.sqrt(2),np.sqrt(2),np.pi/4])

t = points_in_box(xyz,label)
print(t)

[[ 7.07106781e-01  0.00000000e+00 -7.07106781e-01 -2.22044605e-16]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00  1.00000000e+00]
 [ 7.07106781e-01  0.00000000e+00  7.07106781e-01 -2.82842712e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[1 2]


In [117]:
def sample_w_padding(indices, num_needed):
    num_elements = indices.size
    if num_elements >= num_needed:
        return np.random.choice(indices, size=num_needed, replace=False)
    else:
        choice = indices
        repeat = np.random.choice(indices, size=num_needed-num_elements, replace=True)
        return np.concatenate((choice, repeat))


In [195]:
xyz = np.hstack([np.arange(5).reshape(-1,1)]*5)

In [121]:
xyz

array([[0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1],
       [2, 2, 2, 2, 2],
       [3, 3, 3, 3, 3],
       [4, 4, 4, 4, 4]])

In [208]:
ind = [np.array([2, 1, 0, 1, 1, 1]), np.array([1, 0, 3, 3, 0, 0])]
print(ind)

[array([2, 1, 0, 1, 1, 1]), array([1, 0, 3, 3, 0, 0])]


In [209]:
xyz[ind]

/var/folders/bx/vb0z4sks0sl7cc5f1pyvrzfw0000gp/T/ipykernel_79171/1876297654.py:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  xyz[ind]


array([2, 1, 0, 1, 1, 1])

In [131]:
valid = np.full(3, False)
valid

array([False, False, False])

In [132]:
np.concatenate([np.zeros(3), np.ones(3)])

array([0., 0., 0., 1., 1., 1.])

In [174]:
xyz = np.random.rand(16400*100,3)

In [186]:
start = timer()

t = (xyz[:,0] > 0.1) & (xyz[:,0] < 0.9) & (xyz[:,1] > 0.1) & (xyz[:,1] < 0.9) & (xyz[:,2] > 0.1) & (xyz[:,2] < 0.9)
# w = np.where(t)[0]
xyz_sub = xyz[t]

duration = timer() - start
print('indexing duration [ms]:  {:.1f}'.format(duration*1000))

indexing duration [ms]:  55.7


In [189]:
def xxx(box):
    h = box[3]
    w = box[4]
    l = box[5]
    x = box[0]
    y = box[1]
    z = box[2]
    ry = box[6]
    d = np.sqrt(l**2+w**2)

    cos_ry = np.cos(ry)
    sin_ry = np.sin(ry)
    T = [[ cos_ry, 0, sin_ry, x],
        [0,       1, 0,      y],
        [-sin_ry, 0, cos_ry, z]]
    T = np.array(T)
    C = T[:3, :3]
    r = T[:3,  3]
    T_inv = np.zeros((3,4))
    T_inv[:3,:3] = C.T
    T_inv[:3, 3] = np.dot(-C.T, r)

In [187]:
box = np.random.rand(100,7)

In [192]:
start = timer()

xxx(box[0])

duration = timer() - start
print('indexing duration [ms]:  {:.1f}'.format(duration*1000))

indexing duration [ms]:  0.2


In [206]:
l = []
l += [1,2,2]
l += [1,2,2]
l

[1, 2, 2, 1, 2, 2]

In [218]:
a = np.array([[1,1,1],[2,2,2],[3,3,3]])
a[0].reshape(-1,3).shape

(1, 3)

In [221]:
a = np.array([]).reshape(-1,7)

In [224]:
a.shape

(0, 7)

In [230]:
def nms(pred, score, threshold):
    '''
    Task 5
    Implement NMS to reduce the number of predictions per frame with a threshold
    of 0.1. The IoU should be calculated only on the BEV.
    input
        pred (N,7) 3D bounding box with (x,y,z,h,w,l,ry)
        score (N,) confidence scores
        threshold (float) upper bound threshold for NMS
    output
        s_f (M,7) 3D bounding boxes after NMS
        c_f (M,1) corresopnding confidence scores
    '''
    if len(pred) == 0:
        # print("task5 pred shape", pred.shape)
        s_f = np.array([]).reshape(-1,7)
        c_f = np.array([]).reshape(-1,1)
        return s_f, c_f
    return 0

In [231]:
pred = []
nms(pred,0,0)

(array([], shape=(0, 7), dtype=float64),
 array([], shape=(0, 1), dtype=float64))

In [237]:
np.arange(1) != 0

array([False])

In [244]:
def label2corners(label):
    '''
    Task 1
    input
        label (N,7) 3D bounding box with (x,y,z,h,w,l,ry)
    output
        corners (N,8,3) corner coordinates in the rectified reference frame
    '''
    corners = []
    for bbox in label:
        h = bbox[3]
        w = bbox[4]
        l = bbox[5]
        x = bbox[0]
        y = bbox[1]
        z = bbox[2]
        ry = bbox[6]

        '''
              1 -------- 0
             /|         /|
            2 -------- 3 .
            | |        | |
            . 5 -------- 4
            |/         |/
            6 -------- 7
        ''' 
        corner_3d = [[l/2, -h,  w/2],
                    [-l/2, -h,  w/2],
                    [-l/2, -h, -w/2],
                    [ l/2, -h, -w/2],
                    [ l/2,  0,  w/2],
                    [-l/2,  0,  w/2],
                    [-l/2,  0, -w/2],
                    [ l/2,  0, -w/2]]

        corner_3d = np.array(corner_3d)
        corner_3d = np.hstack([corner_3d, np.ones((8,1))])
        corner_3d = corner_3d.T

        cos_ry = np.cos(ry)
        sin_ry = np.sin(ry)
        T = [[ cos_ry, 0, sin_ry, x],
            [0,       1, 0,      y],
            [-sin_ry, 0, cos_ry, z],
            [0,       0, 0,      1]]
        T = np.array(T)

        corner_3d = np.dot(T, corner_3d)
        corner_3d = corner_3d.T
        corner_3d = corner_3d[:, :3]
        corners.append(corner_3d)

    corners = np.array(corners).reshape(-1, 8, 3)
    assert corners.shape == (len(label), 8, 3)

    return corners


def box3d_vol(corners):
    ''' corners: (8,3) no assumption on axis direction '''
    a = np.sqrt(np.sum((corners[0,:] - corners[1,:])**2))
    b = np.sqrt(np.sum((corners[1,:] - corners[2,:])**2))
    c = np.sqrt(np.sum((corners[0,:] - corners[4,:])**2))
    return a*b*c

def get_iou(pred, target):
    '''
    Task 1
    input
        pred (N,7) 3D bounding box corners
        target (M,7) 3D bounding box corners
    output
        iou (N,M) pairwise 3D intersection-over-union
    '''

    corners_pred = label2corners(pred)
    corners_target = label2corners(target)

    IOU = []

    for corner_pred in corners_pred:
        iou_row = []
        for corner_target in corners_target:
            rect1 = [(corner_pred[i,0], corner_pred[i,2]) for i in range(3,-1,-1)]
            rect2 = [(corner_target[i,0], corner_target[i,2]) for i in range(3,-1,-1)] 

            inter_area = 1

            ymin = max(corner_pred[0,1], corner_target[0,1])
            ymax = min(corner_pred[4,1], corner_target[4,1])

            inter_vol = inter_area * max(0.0, ymax-ymin)
            
            vol1 = box3d_vol(corner_pred)
            vol2 = box3d_vol(corner_target)
            iou = inter_vol / (vol1 + vol2 - inter_vol)

            iou_row.append(iou)

        IOU.append(iou_row)

    IOU = np.array(IOU).reshape(len(pred),len(target))
    return IOU

In [246]:
a = get_iou(np.array([]).reshape(-1,7),np.empty((1,7)))

In [250]:
a[:,:]

array([], shape=(0, 1), dtype=float64)